# **Implementing RAG pipeline for querying and extracting relevant information from list of PDF docs**

Due to security/privacy concern over using Chatmodels on sensitive datas, It is not recommended to train the model using private sensitive datas.

Therefore, RAG pipeline is used for extracting relevant infos from sensitive datas, without requiring to train the model again.



In [ ]:
# We will be using langchain framework to build end-to-end pipeline for RAG flow.
# langchain is most popular framework to building LLM usecase pipelines.

!pip install -r requirements.txt

In [55]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import chains, vectorstores
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os
import PyPDF2
import pypdf
import re

# Indexing the Documents and storing it into **VectorStores** Databases.

These documents will be converted into fixed sized chunks and then stored in document embedded forms.

This way, it will be easy to retrieve the required documents.

In [41]:
pdf_document_load = PyPDF2.PdfReader(open("/content/Data_Structures_and_Algorithms_in_Python.pdf", "rb"))

In [42]:
len(pdf_document_load.pages)

770

In [56]:
# Start building pipelines be intiating the model

# load enviroment variables and API_KEYs
load_dotenv()

# Instantiate chat model
def init_chatmodel():
  chatmodel = ChatOpenAI(model="gpt-3.5-turbo")
  return chatmodel

# instantiate Embeddings
def init_embeddings():
  embeddings = OpenAIEmbeddings()
  return embeddings

# load the pdf documents and load its contents for further processing
def load_pdf_document(pdfPath):
  pdf_document = PyPDFLoader(pdfPath)
  documents = pdf_document.load()
  return documents

# split the documents into multiple chunks(of size 1000 tokens) for indexing
def split_pdf_documents(documents):
  doc_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  docs = doc_splitter.split_documents(documents)
  return docs

In [57]:
# now, document embedding is to be done using BERT based OpenAIEmbeddings
"""
This method leverages the power of a OpenAIEmbeddings model for embedding generation
and the efficiency of Chroma for storing and querying embeddings, facilitating
various downstream tasks like document retrieval and clustering.
ChromaDB then adds each document's embedding to the collection using unique
document identifiers
"""

def create_vectorDB_store(textdocs, embeddingFun):
  vectorDB = vectorstores.Chroma.from_documents(textdocs, embeddingFun)
  return vectorDB


In [61]:
# Retrieval Part
"""
Given a query and documents, this method will fetch the relevant documents
based on similarity search.
"""

# It can produce n number of relevant documents chunks requested by user.

def get_retrieved_docs(n_documents, stored_VectorDB):
  retrieved_docs = stored_VectorDB.as_retriever(searchType="similarity", search_kwargs={"k":n_documents})
  return retrieved_docs

# Combining all components to create the RAG pipeline

In [ ]:
# Integrate all components into RAG pipeline for end-to-end chain flow

loaded_docs = load_pdf_document("/content/Data_Structures_and_Algorithms_in_Python.pdf")
textdocs = split_pdf_documents(loaded_docs)
print("chunk size: ", len(textdocs))
vectordb = create_vectorDB_store(textdocs, embeddingFun=init_embeddings())
retrievers = get_retrieved_docs(3, vectordb)
chatmodel = init_chatmodel()

rag_chain = chains.RetrievalQA.from_chain_type(
    llm=chatmodel,
    chain_type="stuff",
    retriever=retrievers,
    return_source_documents=True
)



In [ ]:
# Calling RAG pipeline with query:
query_result1 = rag_chain.invoke({"query": "What are the types of sorting algorithms?"})

print("answer: ", query_result1["result"])

In [ ]:
# We can also know which documents consitutes the answer:
for document in query_result1["source_documents"]:
  print(re.sub(r"\s+", " ", document.page_content.strip()))
  print("\n-------------------------------------------------------\n")

In [ ]:
# Setting up memory context for Chat of previous query.
chat_history1 = [(query_result1["query"], query_result1["result"])]
query_result2 = rag_chain.invoke({
    "query": "Is there any other types of sorting algorithms?",
    "chat_history": chat_history1
})
print("answer: ", query_result2["result"])

In [ ]:
for document in query_result2["source_documents"]:
  print(re.sub(r"\s+", " ", document.page_content.strip()))
  print("\n-------------------------------------------------------\n")